# 食事する哲学者問題の完全解析と実装

## 1. 多角的問題分析

### 競技プログラミング視点
- **並行制御の古典問題**: デッドロック・飢餓・競合状態の回避が必須
- **制約分析**: 5人の哲学者、各1-60回食事、5本の共有フォーク
- **実行速度**: ロック競合の最小化、待機時間の削減
- **メモリ効率**: 固定サイズ(5個のLock)で O(1) 空間

### 業務開発視点
- **スレッドセーフティ**: Pythonの`threading.Lock`による排他制御
- **デッドロック防止**: 複数の戦略から最適なものを選択
- **保守性**: コードの意図が明確で、将来の拡張に対応可能
- **型安全性**: Callable型の正確な指定

### Python特有分析
- **GIL (Global Interpreter Lock)**: I/O待機・ロック待機時は他スレッド実行可能
- **threading.Lock**: C実装による高速な排他制御
- **コンテキストマネージャ**: `with`文による安全なリソース管理
- **CPython最適化**: ネストされた`with`文の効率的な処理

## 2. アルゴリズム比較表

| アプローチ | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | デッドロック回避 | CPython最適化 | 備考 |
|---------|----------|----------|--------------|-------|--------------|------------|------|
| リソース順序付け | O(1) | O(1) | 低 | ★★★ | 完全防止 | 適 | 最も推奨 |
| セマフォ制限(4人) | O(1) | O(1) | 低 | ★★☆ | 完全防止 | 適 | 並行性やや低下 |
| 奇数偶数戦略 | O(1) | O(1) | 低 | ★★☆ | 完全防止 | 適 | 実装シンプル |
| グローバルロック | O(1) | O(1) | 低 | ★☆☆ | 完全防止 | 適 | 並行性なし |

## 3. 採用アルゴリズムと根拠

### 選択: **リソース順序付け戦略**

#### デッドロック発生メカニズムの理解

```
通常アプローチ(全員が左→右):
哲学者0: fork0(左) → fork1(右)
哲学者1: fork1(左) → fork2(右)
哲学者2: fork2(左) → fork3(右)
哲学者3: fork3(左) → fork4(右)
哲学者4: fork4(左) → fork0(右)  ← 循環依存!

全員が左フォークを取得 → 全員が右フォークを待つ → デッドロック
```

#### リソース順序付けによる解決

```
フォークに全順序を定義し、常に小さい番号→大きい番号の順で取得:

哲学者0: min(0,1)=0 → max(0,1)=1
哲学者1: min(1,2)=1 → max(1,2)=2
哲学者2: min(2,3)=2 → max(2,3)=3
哲学者3: min(3,4)=3 → max(3,4)=4
哲学者4: min(4,0)=0 → max(4,0)=4  ← 順序が変わる!

循環依存が発生しない → デッドロック不可能
```

#### 選択理由
1. **数学的保証**: 有向グラフに閉路が形成されない
2. **実装の簡潔性**: 追加の同期機構不要
3. **最大の並行性**: 最大5人が同時に動作可能
4. **Python最適化**: `threading.Lock`と`with`文の効率的活用

## 4. Python特有最適化ポイント

### threading.Lockの特性
- **C実装**: CPythonで高速に動作
- **コンテキストマネージャ**: 例外安全な自動解放
- **再入不可**: 同じスレッドでの再取得でデッドロック（注意）

### with文のネスト最適化
```python
# Python 3.1+: 複数コンテキストマネージャの効率的な記述
with lock1, lock2:
    # 処理
    pass

# 上記は以下と等価だが、よりPythonic
with lock1:
    with lock2:
        # 処理
        pass
```

### GIL考慮事項
- ロック待機中は他スレッドが実行可能
- I/O待機と同様にGILを解放
- この問題では並行性を最大限活用可能

## 5. 実装パターン

### 業務開発版（型安全・可読性重視）

Analyze Complexity
Runtime 85 ms
Beats 72.90%
Memory 20.62 MB
Beats 5.76%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    食事する哲学者問題の解決クラス（業務開発版）
    
    リソース順序付け戦略によりデッドロックを完全防止。
    常に小さいフォーク番号→大きいフォーク番号の順でロック取得することで、
    循環待機を数学的に不可能にする。
    
    フォーク配置:
        哲学者i の左フォーク = i
        哲学者i の右フォーク = (i + 1) % 5
    
    Attributes:
        _forks: 各フォークに対応するLockオブジェクトのリスト
    
    Examples:
        >>> dp = DiningPhilosophers()
        >>> # 5つのスレッドから同時にwantsToEatを呼び出す
    """
    
    def __init__(self) -> None:
        """
        5本のフォークに対応する5つのLockオブジェクトを初期化
        
        Time Complexity: O(1)
        Space Complexity: O(1) - 固定サイズ5
        """
        self._forks: list[Lock] = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        哲学者が食事を行う処理
        
        実装戦略:
        1. 左右のフォークIDを特定
        2. 小さいID→大きいIDの順でロック取得（デッドロック防止）
        3. 両方のロック取得後、必ず左→右の順でpick関数を呼び出す
        4. 食事後、右→左の順でput関数を呼び出す
        5. ロック解放（自動的に逆順で解放される）
        
        Args:
            philosopher: 哲学者のID (0-4)
            pickLeftFork: 左フォークを取る関数（出力記録用）
            pickRightFork: 右フォークを取る関数（出力記録用）
            eat: 食事をする関数（出力記録用）
            putLeftFork: 左フォークを置く関数（出力記録用）
            putRightFork: 右フォークを置く関数（出力記録用）
        
        Time Complexity: O(1) - ロック待機時間を除く
        Space Complexity: O(1)
        
        Thread Safety: 完全にスレッドセーフ、デッドロックなし
        """
        # 左右のフォークIDを計算
        left_fork_id: int = philosopher
        right_fork_id: int = (philosopher + 1) % 5
        
        # リソース順序付け: 小さいID→大きいIDの順でロック
        first_fork_id: int = min(left_fork_id, right_fork_id)
        second_fork_id: int = max(left_fork_id, right_fork_id)
        
        # 両方のロックを順序付けて取得
        with self._forks[first_fork_id]:
            with self._forks[second_fork_id]:
                # 両方のロック取得後、必ず左→右の順でpick
                pickLeftFork()
                pickRightFork()
                
                # 食事
                eat()
                
                # フォークを戻す（右→左の順）
                putRightFork()
                putLeftFork()
```

### 競技プログラミング版（性能最優先）

Analyze Complexity
Runtime 91 ms
Beats 55.40%
Memory 20.62 MB
Beats 5.76%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    競技プログラミング向け最適化実装
    
    Time Complexity: O(1) per call
    Space Complexity: O(1) - 固定5個のLock
    """
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        リソース順序付け戦略による最適化実装
        
        Time: O(1), Space: O(1)
        """
        # フォークID計算
        left, right = philosopher, (philosopher + 1) % 5
        first, second = (left, right) if left < right else (right, left)
        
        # ネストされたwith文でロック取得
        with self._forks[first], self._forks[second]:
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
```

### さらに最適化されたワンライナー版

Analyze Complexity
Runtime 108 ms
Beats 16.79%
Memory 20.53 MB
Beats 10.55%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        l, r = philosopher, (philosopher + 1) % 5
        with self._forks[min(l, r)], self._forks[max(l, r)]:
            pickLeftFork(); pickRightFork(); eat(); putRightFork(); putLeftFork()
```

## 6. 別解: セマフォ制限戦略

Wrong Answer
0 / 24 testcases passed

```python
from typing import Callable
from threading import Lock, Semaphore

class DiningPhilosophersSemaphore:
    """
    セマフォによる同時食事人数制限アプローチ
    
    戦略: 最大4人までしか同時に食事できないように制限。
    5人中4人が食事中の場合、残り1人は必ず両方のフォークを取得可能。
    
    利点: 実装が直感的
    欠点: 並行性がやや低下（最大4人まで）
    """
    
    def __init__(self) -> None:
        self._forks: list[Lock] = [Lock() for _ in range(5)]
        # 最大4人まで同時に食事可能
        self._semaphore: Semaphore = Semaphore(4)
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        セマフォ制限付き食事処理
        
        Time: O(1), Space: O(1)
        """
        left_fork = philosopher
        right_fork = (philosopher + 1) % 5
        
        with self._semaphore:  # 最大4人まで入場
            with self._forks[left_fork]:
                pickLeftFork()
                with self._forks[right_fork]:
                    pickRightFork()
                    eat()
                    putRightFork()
                putLeftFork()
```

## 7. 検証と境界値テスト

### エッジケース確認
```python
# 1. n=1の場合: 各哲学者1回ずつ食事
#    - 実行順序は非決定的だが、全員が食事可能

# 2. n=60の場合（最大制約）: 各哲学者60回食事
#    - デッドロックなし、飢餓なし

# 3. 同一哲学者の連続呼び出し
#    - philosopher=0が連続で呼ばれても正常動作

# 4. 全哲学者が同時に要求
#    - 最悪ケースでも最終的に全員が食事可能
```

### デッドロック防止の数学的証明

**Coffmanの4条件**:
1. ✓ 相互排除: Lockにより保証
2. ✓ 保持待ち: 1つのフォークを持ちながら次を待つ
3. ✓ 非横取り: Lockは強制解放不可
4. ✗ **循環待機**: リソース順序付けにより不可能

**証明**: 
- フォークに全順序 {0 < 1 < 2 < 3 < 4} を定義
- 全哲学者が小→大の順でフォーク取得
- 有向グラフ G=(V,E) で V=哲学者, E=待機関係
- 常に i < j なら 哲学者iが哲学者jを待つ
- ∴ 閉路が存在しない
- ∴ デッドロック不可能 ∎

## 8. Python特有の実装詳細

### threading.Lockの内部動作
```python
# Lockの状態遷移
# unlocked → locked (acquire)
# locked → unlocked (release)

# with文による自動管理
with lock:  # __enter__ で acquire()
    # 処理
    pass  # __exit__ で release()（例外時も確実）
```

### 複数コンテキストマネージャの展開
```python
# Python 3.1+の最適化
with lock1, lock2:
    pass

# 内部的には以下と同等
with lock1:
    with lock2:
        pass

# ただしPython 3.1+では1つのwith文として最適化される
```

### GILとの相互作用
```python
# Lock.acquire()呼び出し時:
# 1. GILを保持したままロック取得を試みる
# 2. ロックが取得できない場合、GILを解放して待機
# 3. ロック取得可能になったら、GILを再取得してロック取得
# 4. 他のスレッドが実行可能になる
```

## まとめ

### 推奨実装: リソース順序付け戦略

**最終推奨コード**:
```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        left, right = philosopher, (philosopher + 1) % 5
        first, second = min(left, right), max(left, right)
        
        with self._forks[first], self._forks[second]:
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
```

### 実装の利点
- ✅ **デッドロック完全防止**: 数学的に証明可能
- ✅ **飢餓なし**: 全哲学者が公平に食事可能
- ✅ **最大並行性**: 最大5人が同時動作
- ✅ **シンプル**: 10行以内の実装
- ✅ **Pythonic**: `with`文による安全なリソース管理
- ✅ **型安全**: Pylance完全対応
- ✅ **保守性**: コードの意図が明確

この実装は、並行制御の古典問題に対する**理論的に正しく、実装的にシンプル**な解決策です。

# 食事する哲学者問題の徹底的な最適化分析

## 1. 現状分析

### パフォーマンス結果の考察

```
業務開発版:     Runtime 85ms  (72.90%) | Memory 20.62MB (5.76%)
競技版:         Runtime 91ms  (55.40%) | Memory 20.62MB (5.76%)
ワンライナー版: Runtime 108ms (16.79%) | Memory 20.53MB (10.55%)
セマフォ版:     Wrong Answer
```

**重要な気づき**:
1. ✅ **業務開発版が最速** - ネストされた`with`の方が効率的
2. ❌ **メモリ使用量が異常に高い** (5.76%は下位) - 通常O(1)のはず
3. ❌ **ワンライナーが最遅** - セミコロン連結はPython非推奨
4. ❌ **セマフォ版がWA** - 実装ミスの可能性

## 2. メモリ使用量の問題分析

### 原因究明

```python
# 現在の実装
with self._forks[first], self._forks[second]:
    pickLeftFork()
    pickRightFork()
    eat()
    putRightFork()
    putLeftFork()
```

**問題点**:
- 複数の`with`文は内部でスタックフレームを消費
- 関数呼び出しが5回（オーバーヘッド）
- LeetCodeの測定方法の特性（スレッド関連のオーバーヘッド）

### メモリ最適化戦略

1. **不要な変数を削減**
2. **関数呼び出しを最小化**（できない - 仕様上必須）
3. **`with`文のネスト方法を変更**

## 3. 最適化実装

### 最高性能版（メモリ・速度最適化）

Analyze Complexity
Runtime 82 ms
Beats 79.14%
Memory 20.50 MB
Beats 10.55%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    最適化版: メモリとランタイムの両方を最適化
    
    最適化ポイント:
    1. 不要な中間変数の削除
    2. min/maxの呼び出しを1回に削減
    3. 明示的なネストでスタックフレーム削減
    """
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        Time: O(1), Space: O(1)
        最小のメモリフットプリントを実現
        """
        left = philosopher
        right = (philosopher + 1) % 5
        
        # 小さい方を先にロック（デッドロック回避）
        if left < right:
            self._forks[left].acquire()
            self._forks[right].acquire()
        else:
            self._forks[right].acquire()
            self._forks[left].acquire()
        
        try:
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
        finally:
            # 解放順序は取得の逆
            if left < right:
                self._forks[right].release()
                self._forks[left].release()
            else:
                self._forks[left].release()
                self._forks[right].release()
```

### 超最適化版（メモリ最小）

Analyze Complexity
Runtime 87 ms
Beats 67.63%
Memory 20.48 MB
Beats 13.91%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    メモリ使用量を極限まで削減
    中間変数を可能な限り排除
    """
    
    __slots__ = ('_forks',)  # メモリオーバーヘッド削減
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """最小メモリフットプリント実装"""
        right = (philosopher + 1) % 5
        
        # 条件分岐で中間変数を削減
        if philosopher < right:
            self._forks[philosopher].acquire()
            self._forks[right].acquire()
            try:
                pickLeftFork()
                pickRightFork()
                eat()
                putRightFork()
                putLeftFork()
            finally:
                self._forks[right].release()
                self._forks[philosopher].release()
        else:
            self._forks[right].acquire()
            self._forks[philosopher].acquire()
            try:
                pickLeftFork()
                pickRightFork()
                eat()
                putRightFork()
                putLeftFork()
            finally:
                self._forks[philosopher].release()
                self._forks[right].release()
```

### ハイブリッド最適化版（推奨）

Analyze Complexity
Runtime 112 ms
Beats 11.03%
Memory 20.64 MB
Beats 5.76%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    速度とメモリのバランス最適化版
    
    - acquire/releaseの明示的制御でオーバーヘッド削減
    - 最小限の中間変数
    - __slots__によるメモリ削減
    """
    
    __slots__ = ('_forks',)
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        ハイブリッド最適化実装
        Time: O(1), Space: O(1)
        """
        # 右フォークIDのみ計算（philosopher=左フォーク）
        right = (philosopher + 1) % 5
        
        # リソース順序付け: 小さいID優先
        if philosopher < right:
            # 通常ケース: 左(小) → 右(大)
            self._forks[philosopher].acquire()
            self._forks[right].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[right].release()
            self._forks[philosopher].release()
        else:
            # 哲学者4のケース: 右(小) → 左(大)
            self._forks[right].acquire()
            self._forks[philosopher].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[philosopher].release()
            self._forks[right].release()
```

## 4. セマフォ版のバグ修正

Analyze Complexity
Runtime 118 ms
Beats 7.19%
Memory 20.56 MB
Beats 10.55%

### Wrong Answerの原因

```python
# 元のコード（バグあり）
with self._semaphore:
    with self._forks[left_fork]:
        pickLeftFork()
        with self._forks[right_fork]:
            pickRightFork()
            eat()
            putRightFork()
        putLeftFork()  # ← これが間違い！
```

**問題**: `putLeftFork()`がleft_forkのロック外で呼ばれている

### 修正版セマフォ実装

```python
from typing import Callable
from threading import Lock, Semaphore

class DiningPhilosophers:
    """修正版セマフォ戦略"""
    
    __slots__ = ('_forks', '_semaphore')
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
        self._semaphore = Semaphore(4)
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """修正されたセマフォ実装"""
        left = philosopher
        right = (philosopher + 1) % 5
        
        self._semaphore.acquire()
        self._forks[left].acquire()
        self._forks[right].acquire()
        
        pickLeftFork()
        pickRightFork()
        eat()
        putRightFork()
        putLeftFork()
        
        self._forks[right].release()
        self._forks[left].release()
        self._semaphore.release()
```

## 5. 究極の最適化版（推奨実装）

Analyze Complexity
Runtime 83 ms
Beats 77.22%
Memory 20.68 MB
Beats 5.76%

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    究極最適化版 - LeetCode最高性能を目指す
    
    最適化技術:
    1. __slots__: インスタンス辞書オーバーヘッド削減
    2. 明示的acquire/release: with文のオーバーヘッド削減
    3. 最小限の変数: スタックフレーム削減
    4. 条件分岐最小化: 分岐予測最適化
    """
    
    __slots__ = ('_forks',)
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        究極最適化実装
        
        Time Complexity: O(1)
        Space Complexity: O(1) - 追加メモリなし
        """
        # 右フォークのみ計算（philosopher自身が左フォークID）
        r = (philosopher + 1) % 5
        
        # philosopher < r の場合が多い（4/5のケース）
        # 分岐予測を考慮して頻度の高い方を先に
        if philosopher < r:
            # ケース1-4: 哲学者0,1,2,3
            self._forks[philosopher].acquire()
            self._forks[r].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[r].release()
            self._forks[philosopher].release()
        else:
            # ケース5: 哲学者4のみ
            self._forks[r].acquire()
            self._forks[philosopher].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[philosopher].release()
            self._forks[r].release()
```

## 6. パフォーマンス比較と選択基準

### 最適化技術の効果

| 最適化技術 | ランタイム改善 | メモリ改善 | 実装複雑度 |
|---------|------------|----------|----------|
| `__slots__` | - | ★★★ | 低 |
| 明示的acquire/release | ★★ | ★ | 中 |
| 中間変数削減 | ★ | ★★ | 低 |
| with文削除 | ★★ | ★★ | 中 |
| 条件分岐最適化 | ★ | - | 低 |

### 推奨実装の選択

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    最終推奨実装
    Runtime目標: 70-80ms (Top 80%)
    Memory目標: 20.5MB (Top 20%)
    """
    
    __slots__ = ('_forks',)
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        r = (philosopher + 1) % 5
        
        if philosopher < r:
            self._forks[philosopher].acquire()
            self._forks[r].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[r].release()
            self._forks[philosopher].release()
        else:
            self._forks[r].acquire()
            self._forks[philosopher].acquire()
            pickLeftFork()
            pickRightFork()
            eat()
            putRightFork()
            putLeftFork()
            self._forks[philosopher].release()
            self._forks[r].release()
```

## 7. さらなる最適化の可能性

### 考慮すべき高度な技術

```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    """
    極限最適化: インライン展開とアンロール
    """
    
    __slots__ = ('_forks',)
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        """
        完全最適化版
        - 関数呼び出しオーバーヘッド最小化
        - 分岐予測最適化
        """
        # 条件式を使って計算を統一
        r = (philosopher + 1) % 5
        first = philosopher if philosopher < r else r
        second = r if philosopher < r else philosopher
        
        # 統一されたロック取得
        self._forks[first].acquire()
        self._forks[second].acquire()
        
        # 処理
        pickLeftFork()
        pickRightFork()
        eat()
        putRightFork()
        putLeftFork()
        
        # 統一されたロック解放
        self._forks[second].release()
        self._forks[first].release()
```

## まとめ

### 最終推奨実装

**ランタイム優先版**（85ms → 70-75ms目標）:
```python
from typing import Callable
from threading import Lock

class DiningPhilosophers:
    __slots__ = ('_forks',)
    
    def __init__(self) -> None:
        self._forks = [Lock() for _ in range(5)]
    
    def wantsToEat(
        self,
        philosopher: int,
        pickLeftFork: Callable[[], None],
        pickRightFork: Callable[[], None],
        eat: Callable[[], None],
        putLeftFork: Callable[[], None],
        putRightFork: Callable[[], None]
    ) -> None:
        r = (philosopher + 1) % 5
        if philosopher < r:
            self._forks[philosopher].acquire()
            self._forks[r].acquire()
            pickLeftFork(); pickRightFork(); eat(); putRightFork(); putLeftFork()
            self._forks[r].release()
            self._forks[philosopher].release()
        else:
            self._forks[r].acquire()
            self._forks[philosopher].acquire()
            pickLeftFork(); pickRightFork(); eat(); putRightFork(); putLeftFork()
            self._forks[philosopher].release()
            self._forks[r].release()
```

この実装で **Runtime 70-80ms (Top 80-90%)、Memory 20.5MB (Top 15-20%)** を達成できるはずです！